In [1]:
import numpy as np
import pandas as pd
import torch 

seed=23
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
df = pd.read_csv('./dataset/train.csv')
test_df = pd.read_csv('./dataset/test.csv')


In [3]:
X_df, Y_df = df.iloc[:,1:-1], df.iloc[:,-1]
ids, Xtest_df = test_df.iloc[:,0], test_df.iloc[:,1:]


In [4]:
X, Y = X_df.to_numpy(), Y_df.to_numpy()
X_test = Xtest_df.to_numpy()

In [5]:
"""
Convert string classes to integer labels
"""

classes = np.unique(Y)
class_to_idx = {classes[i]: i for i in range(len(classes))}
idx_to_class = {class_to_idx[cl]: cl for cl in class_to_idx.keys()}
Y = np.array([class_to_idx[y] for y in Y])

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val= train_test_split(
    X, Y, test_size=0.2, random_state=seed)

/Users/kartiksinghal/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
"""
Outlier detection
"""
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=10)
inlier_pred_labels = clf.fit_predict(X_train)
X_train = X_train[inlier_pred_labels == 1]
Y_train = Y_train[inlier_pred_labels == 1]

In [8]:
"""
Standardising the train data
"""

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [9]:
import imblearn

oversample = imblearn.over_sampling.SMOTE()
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

In [10]:
"""
Feature engineering
"""

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# pca = PCA(n_components=64)
# pca.fit(X_train)

# X_train = pca.transform(X_train)

lda = LinearDiscriminantAnalysis(n_components=19)
lda.fit(X_train, Y_train)

X_train = np.c_[X_train, lda.transform(X_train)]
X_val = np.c_[X_val, lda.transform(X_val)]
X_test = np.c_[X_test, lda.transform(X_test)]

In [11]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=seed, n_init="auto").fit(X_train)
labels = kmeans.predict(X_train)
kmeans_classes = kmeans.labels_

onehot_encoded_array = np.zeros((labels.size, kmeans_classes.max()+1), dtype=int)
onehot_encoded_array[np.arange(labels.size),labels] = 1 
X_train = np.c_[X_train, onehot_encoded_array] 

labels = kmeans.predict(X_val)
onehot_encoded_array = np.zeros((labels.size, kmeans_classes.max()+1), dtype=int)
onehot_encoded_array[np.arange(labels.size),labels] = 1 
X_val = np.c_[X_val, onehot_encoded_array] 

labels = kmeans.predict(X_test)
onehot_encoded_array = np.zeros((labels.size, kmeans_classes.max()+1), dtype=int)
onehot_encoded_array[np.arange(labels.size),labels] = 1 
X_test = np.c_[X_test, onehot_encoded_array] 

In [12]:
from sklearn.linear_model import LogisticRegression

models = []
for cls in classes:
    model = LogisticRegression()
    class_Y_train = np.where(Y_train == class_to_idx[cls], 1, 0)
    class_Y_val = np.where(Y_val == class_to_idx[cls], 1, 0)
    model.fit(X_train, class_Y_train)
    val_score = model.score(X_val, class_Y_val)
    print(val_score)
    models.append(model)

val_preds = []
for model in models:
    pred = model.predict(X_val)
    val_preds.append(pred)

test_preds = []
for model in models:
    pred = model.predict(X_test)
    test_preds.append(pred)


/Users/kartiksinghal/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8688524590163934


/Users/kartiksinghal/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8770491803278688


/Users/kartiksinghal/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/Users/kartiksinghal/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9098360655737705


/Users/kartiksinghal/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9631147540983607
0.9590163934426229


/Users/kartiksinghal/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
np.array(val_preds).shape

[array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 0]),
 array([0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1,

In [14]:
(np.array(val_preds).argmax(axis=0) == Y_val).mean()

0.4016393442622951

In [ ]:
class_index = np.array(preds).argmax(axis=0)
